# Proyecto Sprint 9 Machine Learning
Por Rodrigo Ochoa

Megaline busca mejorar la eficiencia de sus ofertas de planes migrando a los clientes de planes antiguos. Este proyecto desarrollará un modelo de aprendizaje automático para predecir el plan óptimo ('Smart' o 'Ultra') para cada cliente basándose en su comportamiento de uso. El objetivo es proporcionar a Megaline una herramienta que maximice la satisfacción del cliente y los ingresos.

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#Inspección inicial
df = pd.read_csv('/datasets/users_behavior.csv')

print(df.head())
print(df.info())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [3]:
# Esta sección divide el conjunto de datos en conjuntos de entrenamiento, validación y prueba.
# La división adecuada es esencial para la evaluación del modelo y evita la fuga de datos.
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=42)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

print("Train size:", len(df_train))
print("Valid size:", len(df_valid))
print("Test size:", len(df_test))

Train size: 1928
Valid size: 643
Test size: 643


In [4]:
# Esta sección entrena y evalúa diferentes modelos de clasificación
# y selecciona el mejor según la precisión del conjunto de validación.
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

best_model = None
best_accuracy = 0

# A. Random Forest
# Esta parte del código entrena y evalúa el clasificador de Bosque Aleatorio.
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=42)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

print("Best Random Forest Accuracy:", best_accuracy)

Best Random Forest Accuracy: 0.8133748055987559


In [5]:
# B. Decision Tree
# Esta parte entrena y evalúa el clasificador de Árbol de Decisión.
best_tree_model = None
best_tree_accuracy = 0
for depth in range (1, 11):
    model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > best_tree_accuracy:
      best_tree_model = model
      best_tree_accuracy = accuracy

print("Best Decision Tree Accuracy:", best_tree_accuracy)

Best Decision Tree Accuracy: 0.7962674961119751


In [6]:
# C. Regresión Logística (Logistic Regression)
# Esta parte entrena y evalúa el modelo de Regresión Logística.
model = LogisticRegression(solver='liblinear', random_state=42)  # Added Logistic Regression
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = model

print("Best Logistic Regression Accuracy:", best_accuracy)

Best Logistic Regression Accuracy: 0.8133748055987559


In [7]:
# 4. Comprobar la calidad en el conjunto de prueba
# Esta sección evalúa el mejor modelo seleccionado en el conjunto de prueba.
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

test_accuracy = best_model.score(features_test, target_test)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.8195956454121306


In [8]:
print("\nBest Model:", best_model.__class__.__name__) 


Best Model: RandomForestClassifier


In [9]:
# 5. Prueba de Cordura (Sanity Check)
# Esta sección realiza una prueba de cordura comparando el rendimiento del modelo
# con un DummyClassifier de línea base que predice la clase más frecuente.
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=42)
dummy_clf.fit(features_train, target_train)
dummy_accuracy = dummy_clf.score(features_test, target_test)

print("Dummy Accuracy:", dummy_accuracy)

if test_accuracy < dummy_accuracy:
    print("Model is worse than dummy - check for errors!")
else:
    print("Model is better than dummy - looks good!")

Dummy Accuracy: 0.6967340590979783
Model is better than dummy - looks good!


Para abordar la necesidad de Megaline de mejorar la eficiencia de sus ofertas de planes migrando a los clientes de planes antiguos, este proyecto exploró modelos de clasificación de aprendizaje automático. El análisis reveló que RandomForestClassifier y LogisticRegression fueron los más efectivos para predecir las preferencias de planes de los clientes, ambos logrando aproximadamente un 81.34% de precisión en el conjunto de validación. El modelo RandomForestClassifier fue elegido para la evaluación final y demostró una precisión de aproximadamente el 82% en el conjunto de prueba, en comparación con una precisión del modelo dummy de alrededor del 70%. Esto indica que el modelo puede identificar con precisión los patrones de comportamiento del usuario que indican la idoneidad del plan. La implementación de este modelo podría permitir a Megaline automatizar las actualizaciones de planes, lo que resultaría en una asignación de recursos más eficiente, una reducción en la pérdida de clientes y una mayor generación de ingresos para Megaline.